In [0]:
!pip install git+https://github.com/fastai/fastai.git
!pip install kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"iamkartik","key":"a723f8654b75ed6d2891131c0b2d3218"}'}

In [3]:
!mkdir ~/.kaggle
!cp /content/kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

 95% 257M/271M [00:03<00:00, 72.5MB/s]
100% 271M/271M [00:03<00:00, 89.0MB/s]
 96% 523M/544M [00:06<00:00, 92.5MB/s]
100% 544M/544M [00:06<00:00, 93.2MB/s]
  0% 0.00/111k [00:00<?, ?B/s]
100% 111k/111k [00:00<00:00, 100MB/s]


In [0]:
!unzip -q train.zip
!unzip -q test.zip

In [7]:
!ls

kaggle.json  sample_submission.csv  test.zip  train.zip
sample_data  test		    train


In [0]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [8]:
imglist = os.listdir('train')
imglist[:10]

['cat.192.jpg',
 'cat.2439.jpg',
 'dog.8348.jpg',
 'cat.8913.jpg',
 'cat.10684.jpg',
 'dog.1640.jpg',
 'dog.6959.jpg',
 'cat.7729.jpg',
 'dog.11825.jpg',
 'dog.12006.jpg']

In [11]:
len(imglist)

2

In [0]:
imgname = imglist[0]

In [22]:
imgname

'dog.1306.jpg'

In [25]:
imgname[:3]

f'train/{imgname}'

'train/dog.1306.jpg'

In [0]:
import shutil
PATH ='/content/train/'


for name in imglist:
  class_name = name[:3]
  #print(os.path.exists(f'{PATH}{class_name}'))
  if os.path.exists(f'{PATH}{class_name}') is False:
    #print('Hello')
    os.mkdir(f'{PATH}{class_name}')
    
  shutil.move(f'{PATH}{name}',f'{PATH}{class_name}/{name}') 
  

In [0]:

sz = 224

In [11]:
cats = os.listdir(f'{PATH}cat')
dogs = os.listdir(f'{PATH}dog')

len(cats),len(dogs)

(12500, 12500)

In [13]:
# moving 2500 (20% ) of images in validation folder

10000.0

In [0]:
os.mkdir('valid')
os.mkdir('valid/cat')
os.mkdir('valid/dog')


i=1
for cat,dog in zip(cats,dogs): 
  if(i==2500):
    break
  
  
  shutil.move(f'train/cat/{cat}',f'valid/cat/{cat}') 
  shutil.move(f'train/dog/{dog}',f'valid/dog/{dog}')
  i+=1
  

In [0]:
tcat = os.listdir(f'{PATH}cat')
vcat = os.listdir('valid/cat')

tdog = os.listdir(f'{PATH}dog')
vdog = os.listdir('valid/dog')

In [14]:
len(tcat),len(vcat),len(tdog),len(vdog)

(10001, 2499, 10001, 2499)

In [0]:
PATH='/content/'

In [16]:
arch = resnet50
data = ImageClassifierData.from_paths(PATH,tfms=tfms_from_model(arch,sz),test_name='test')
learn = ConvLearner.pretrained(arch,data,precompute=True)
learn.fit(0.01,3)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.torch/models/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:03<00:00, 29453665.19it/s]


100%|██████████| 196/196 [02:02<00:00,  2.41it/s]


epoch      trn_loss   val_loss   accuracy   
    0      0.03389    0.023888   0.990996  
    1      0.027961   0.022602   0.992597  
    2      0.028211   0.026702   0.990996  



[array([0.0267]), 0.9909963985594238]

In [0]:
preds = learn.predict(is_test=True)

In [18]:
preds[0]

array([-0.00011, -9.15562], dtype=float32)

In [19]:
# log loss 
log_pred = np.argmax(preds,axis=1)
probs =  np.exp(log_pred)
probs[0]

1.0

In [34]:
probs = np.exp(preds[:,1])   
probs[0]

0.00010562426

In [21]:
probs[:10],preds[:10]

(array([1.     , 1.     , 2.71828, 2.71828, 1.     , 2.71828, 1.     , 2.71828, 2.71828, 1.     ]),
 array([[ -0.00011,  -9.15562],
        [ -0.00004, -10.13893],
        [ -9.07001,  -0.00011],
        [ -8.76486,  -0.00016],
        [ -0.0004 ,  -7.83482],
        [ -8.92847,  -0.00013],
        [ -0.0001 ,  -9.24626],
        [-16.02932,   0.     ],
        [-12.79627,  -0.     ],
        [ -0.00022,  -8.41111]], dtype=float32))

In [0]:
sample = pd.read_csv('sample_submission.csv')

In [45]:
preds[:,1]


array([ -9.15562, -10.13893,  -0.00011, ...,  -7.98055,  -0.0002 ,  -9.67759], dtype=float32)

In [24]:
sample.head()

,id,label
0,1,0.5
1,2,0.5
2,3,0.5
3,4,0.5
4,5,0.5


In [25]:
testlist = os.listdir('test')
testlist[:10]

['3233.jpg',
 '8604.jpg',
 '11763.jpg',
 '10574.jpg',
 '407.jpg',
 '7430.jpg',
 '3549.jpg',
 '10458.jpg',
 '826.jpg',
 '2370.jpg']

In [0]:
testlist=list(map(lambda x:int(x[:-4]),testlist))

In [27]:
x = testlist[0]
x

3233

In [0]:
preds = list(map(lambda x:np.argmax(x),preds))

In [28]:
log_pred[0]

0

In [0]:
subm = pd.DataFrame({'id':testlist,'label':probs})

In [36]:
subm.head()

,id,label
0,3233,0.000106
1,8604,0.000040
2,11763,0.999885
3,10574,0.999844
4,407,0.000396


In [0]:
subm.to_csv('submission_1.csv',index=False)


In [38]:
!kaggle competitions submit -c dogs-vs-cats-redux-kernels-edition -f submission_1.csv -m"first kaggle submission "

Successfully submitted to Dogs vs. Cats Redux: Kernels Edition